In [1]:
import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime, timedelta
import sys
sys.path.append('../../../')
from env import pg_conn

In [62]:
# today = datetime.strptime('05/02/24 00:00:00', '%d/%m/%y %H:%M:%S')  #use this for backdating
today = datetime.today() #use this for regular run
day = today.weekday()
print(day)
file_date = today.strftime('%Y%m%d')
print(f"File date: {file_date}")


0
File date: 20240226


In [63]:
file_end = today.date()
file_start = file_end - timedelta(days=7)

# # manual overwrites below
# file_end = '2024-01-25'
file_start = '2024-01-25' 

print(f"File Dates: {file_start} - {file_end}")
print(type(file_start))

File Dates: 2024-01-25 - 2024-02-26
<class 'str'>


In [64]:
def read_query(database, query):
    engine = create_engine(pg_conn+str(database))
    conn = engine.connect()
    transaction = conn.begin()
    try:
        # perform some database operations here
        df = pd.read_sql(query, conn)
        transaction.commit()
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also    
            print(df)
        return df
    except SQLAlchemyError as e:
        transaction.rollback()
        print(f"Error occurred: {e}")
    finally:
        conn.close()

### Getting all matched transactions from matched_transaction table
matched_transactions for viator over selected period



In [68]:
harmonia_matched_query = f"""SELECT to_Char(spend_amount/100, 'FM9999999990.00') AMOUNT , transaction_date TRANSACTION_DATE, auth_code::varchar(8) AUTH, mid MID, last_four::varchar(4) LAST_FOUR FROM export_transaction pt
                        where provider_slug = 'bpl-viator' and created_at >= '{file_start}' and created_at < '{file_end}';"""
harmonia_matched = read_query("harmonia", harmonia_matched_query)
harmonia_matched


      amount    transaction_date    auth        mid last_four
0     122.00 2024-01-31 05:03:32  031432  020442950      9838
1       1.00 2024-01-25 05:25:07  025983  020150514      1515
2       1.00 2024-01-25 14:50:19  025494  020150514      6841
3      -1.00 2024-01-25 14:50:23  025494  020150514      6841
4      60.00 2024-01-25 17:10:19  025224  020150514      1663
5      72.00 2024-01-26 08:43:04  026069  020150514      1471
6     170.00 2024-01-26 15:06:09  026900  020150514      4938
7       1.00 2024-01-26 18:33:26  026649  020150514      3176
8      95.00 2024-01-26 23:22:44  026097  020150514      8388
9      -1.00 2024-01-28 09:37:31  029746  020150514      8638
10      1.00 2024-01-28 09:37:30  029746  020150514      8638
11    158.00 2024-02-02 10:03:55  002770  020150514      8435
12     -1.00 2024-02-02 13:49:21  002346  020150514      2011
13      1.00 2024-02-02 13:49:20  002346  020150514      2011
14     -1.00 2024-02-02 15:36:10  002303  020150514      0216
15      

,amount,transaction_date,auth,mid,last_four
0,122.00,2024-01-31 05:03:32,031432,020442950,9838
1,1.00,2024-01-25 05:25:07,025983,020150514,1515
2,1.00,2024-01-25 14:50:19,025494,020150514,6841
3,-1.00,2024-01-25 14:50:23,025494,020150514,6841
4,60.00,2024-01-25 17:10:19,025224,020150514,1663
...,...,...,...,...,...
128,-1.00,2024-02-15 15:08:23,015120,020150514,5762
129,-1.00,2024-02-16 09:00:29,016319,020150514,0216
130,367.00,2024-02-16 14:03:31,016320,020150514,0216
131,624.00,2024-02-17 18:01:49,019694,020150514,1350


### Sort data type final file for output


In [69]:
final_file = harmonia_matched.astype({'amount': 'str', 'transaction_date': 'datetime64[ns]', 'auth': 'str', 'mid': 'str', 'last_four': 'str'})
final_file

,amount,transaction_date,auth,mid,last_four
0,122.00,2024-01-31 05:03:32,031432,020442950,9838
1,1.00,2024-01-25 05:25:07,025983,020150514,1515
2,1.00,2024-01-25 14:50:19,025494,020150514,6841
3,-1.00,2024-01-25 14:50:23,025494,020150514,6841
4,60.00,2024-01-25 17:10:19,025224,020150514,1663
...,...,...,...,...,...
128,-1.00,2024-02-15 15:08:23,015120,020150514,5762
129,-1.00,2024-02-16 09:00:29,016319,020150514,0216
130,367.00,2024-02-16 14:03:31,016320,020150514,0216
131,624.00,2024-02-17 18:01:49,019694,020150514,1350


### Generate final file for output

fill in todays date for the file delivery
file dumped in repo make sure to remove before committing code back into repo

In [70]:
#LOOOK AT DATA TYPES!!!!!!!
harmonia_matched.to_csv(f'~/python/snowdrops/data/viator_weekly_files/viator_weekly_transactions_{file_date}.csv',index=False)


In [ ]:
from azure.storage.fileshare import ShareFileClient, ShareDirectoryClient, ShareClient
from env import connection_string



#Create connection to export fileshare
print('Creating connections to azure storage...')
connection_string = connection_string
viator_dir = ShareDirectoryClient.from_connection_string(conn_str=connection_string, share_name="stored", directory_path=dir_name)


#Check if test results have passed and then upload files
print('Checking test results and uploading files...')
file = f'~/python/snowdrops/data/viator_weekly_files/viator_weekly_transactions_{file_date}.csv'
with open(file, "rb") as outfile: 
    viator_dir.upload_file(file, outfile)
    print(f'{file} succesfully uploaded to blob')